In [ ]:
from ckks.ckks_parameters import CKKSParameters
from ckks.ckks_key_generator import CKKSKeyGenerator
from util.polynomial import Polynomial
from ckks.ckks_decryptor import CKKSDecryptor
from ckks.ckks_encryptor import CKKSEncryptor
from ckks.ckks_evaluator import CKKSEvaluator
from util.plaintext import Plaintext
from ckks.ckks_encoder import CKKSEncoder
import numpy as np

# Test 1: CKKS Roundrip

In [2]:
# Test setup
from ckks.ckks_encoder import CKKSEncoder


poly_degree = 8192 # Must be a power of 2
ciph_modulus = 1 << 40
big_modulus = 1 << 1200 # Used for bootstrapping
scaling_factor = 1 << 30
params = CKKSParameters(poly_degree=poly_degree,
                        ciph_modulus=ciph_modulus,
                        big_modulus=big_modulus,
                        scaling_factor=scaling_factor,
                        prime_size=30)

# Message length must be half of poly_degree
message = [4.0 + 0j] * (poly_degree // 2)
print("Message length:", len(message))
print("First 5 elements:", message[:5])


encoder = CKKSEncoder(params) # Contains encode and decode functions
poly = encoder.encode(message, params.scaling_factor)


key_generator = CKKSKeyGenerator(params)
public_key = key_generator.public_key
secret_key = key_generator.secret_key

Message length: 4096
First 5 elements: [(4+0j), (4+0j), (4+0j), (4+0j), (4+0j)]


In [3]:
encryptor = CKKSEncryptor(params, public_key, secret_key)
encrypted_poly = encryptor.encrypt(poly)

decryptor = CKKSDecryptor(params, secret_key)
decrypted_poly = decryptor.decrypt(encrypted_poly)

decoded_message = encoder.decode(decrypted_poly)
print("Decoded message length:", len(decoded_message))
print("First 5 elements of decoded message:", decoded_message[:5])


assert len(decoded_message) == len(message), "Decoded message has incorrect length"
print("Decoded message matches original within tolerance:", np.allclose(decoded_message, message, atol=1e-1, rtol=1e-1))


PARALLEL!!!!!!
PARALLEL!!!!!!
PARALLEL!!!!!!
Decoded message length: 4096
First 5 elements of decoded message: [(4.000000185075158+1.933956379611621e-06j), (4.000004139451005-3.867820841370243e-06j), (4.000006800113919+3.083109651595119e-06j), (3.9999949664073573-3.3080030814388084e-06j), (3.9999924508016846-8.00470612034522e-07j)]
Decoded message matches original within tolerance: True


# Test 2: Addition Is Homomorphic

In [4]:
# Test setup

poly_degree = 8192 # Must be a power of 2
ciph_modulus = 1 << 40
big_modulus = 1 << 1200 # Used for bootstrapping
scaling_factor = 1 << 30
params = CKKSParameters(poly_degree=poly_degree,
                        ciph_modulus=ciph_modulus,
                        big_modulus=big_modulus,
                        scaling_factor=scaling_factor,
                        prime_size=30)

# Message length must be half of poly_degree
message1 = [4.0 + 0j] * (poly_degree // 2)
message2 = [3.0 + 0j] * (poly_degree // 2)
print("Message1 length:", len(message1))
print("Message2 length:", len(message2))
print("First 5 elements of message1:", message1[:5])
print("First 5 elements of message2:", message2[:5])



key_generator = CKKSKeyGenerator(params)


Message1 length: 4096
Message2 length: 4096
First 5 elements of message1: [(4+0j), (4+0j), (4+0j), (4+0j), (4+0j)]
First 5 elements of message2: [(3+0j), (3+0j), (3+0j), (3+0j), (3+0j)]


In [5]:
public_key = key_generator.public_key
secret_key = key_generator.secret_key

encoder = CKKSEncoder(params) # Contains encode and decode functions
plain_poly1 = encoder.encode(message1, params.scaling_factor)
plain_poly2 = encoder.encode(message2, params.scaling_factor)


encryptor = CKKSEncryptor(params, public_key, secret_key)
encrypted_poly1 = encryptor.encrypt(plain_poly1)
encrypted_poly2 = encryptor.encrypt(plain_poly2)

evaluator = CKKSEvaluator(params)
encrypted_message_sum = evaluator.add(encrypted_poly1, encrypted_poly2)

decryptor = CKKSDecryptor(params, secret_key)
decrypted_message_sum = decryptor.decrypt(encrypted_message_sum)

decoded_message_sum = encoder.decode(decrypted_message_sum)

true_message_sum = [m1 + m2 for m1, m2 in zip(message1, message2)]
print("True message sum length:", len(true_message_sum))
print("First 5 elements of true sum:", true_message_sum[:5])
print("First 5 elements of decoded sum:", decoded_message_sum[:5])



assert len(decoded_message_sum) == len(true_message_sum), "Decoded message has incorrect length"
print("Addition is homomorphic:", np.allclose(decoded_message_sum, true_message_sum,  rtol=1e-2, atol=1e-1))  

PARALLEL!!!!!!
PARALLEL!!!!!!
PARALLEL!!!!!!
PARALLEL!!!!!!
PARALLEL!!!!!!
True message sum length: 4096
First 5 elements of true sum: [(7+0j), (7+0j), (7+0j), (7+0j), (7+0j)]
First 5 elements of decoded sum: [(6.999989149448134-4.000147575452434e-06j), (7.0000019968668665-1.8594009908065413e-07j), (6.999998372754779+1.64102787474478e-06j), (6.9999998979573785-3.1251035289926032e-06j), (6.9999995124693974+2.091162182569466e-07j)]
Addition is homomorphic: True


# Test 3: Multiplication Is Homomorphic

In [6]:
# Test setup
poly_degree = 8192 # Must be a power of 2
# NOTE: ciph_modulus of 1 << 40 is too small and produces innacurate results
ciph_modulus = 1 << 800
big_modulus = 1 << 1200
scaling_factor = 1 << 30
params = CKKSParameters(poly_degree=poly_degree,
                        ciph_modulus=ciph_modulus,
                        big_modulus=big_modulus,
                        scaling_factor=scaling_factor,
                        prime_size=30)
# Message length must be half of poly_degree
message1 = [4.0 + 0j] * (poly_degree // 2)
message2 = [3.0 + 0j] * (poly_degree // 2)
print("Message1 length:", len(message1))
print("Message2 length:", len(message2))
print("First 5 elements of message1:", message1[:5])
print("First 5 elements of message2:", message2[:5])



poly_product = plain_poly1.poly.multiply_naive(plain_poly2.poly) # NOTE: for some reason modding using ciph_modulus doesnt work
plain_poly_product = Plaintext(poly_product, params.scaling_factor)

key_generator = CKKSKeyGenerator(params)

Message1 length: 4096
Message2 length: 4096
First 5 elements of message1: [(4+0j), (4+0j), (4+0j), (4+0j), (4+0j)]
First 5 elements of message2: [(3+0j), (3+0j), (3+0j), (3+0j), (3+0j)]


In [1]:

public_key = key_generator.public_key
secret_key = key_generator.secret_key
relin_key = key_generator.relin_key

encoder = CKKSEncoder(params) # Contains encode and decode functions
plain_poly1 = encoder.encode(message1, params.scaling_factor)
plain_poly2 = encoder.encode(message2, params.scaling_factor)



encryptor = CKKSEncryptor(params, public_key, secret_key)
encrypted_poly1 = encryptor.encrypt(plain_poly1)
encrypted_poly2 = encryptor.encrypt(plain_poly2)

evaluator = CKKSEvaluator(params)
encrypted_poly_product = evaluator.multiply(encrypted_poly1, encrypted_poly2, relin_key)

decryptor = CKKSDecryptor(params, secret_key)
decrypted_poly_product = decryptor.decrypt(encrypted_poly_product)

decoded_message_product = encoder.decode(decrypted_poly_product)
print("Decoded message product length:", len(decoded_message_product))
print("First 5 elements of decoded product:", decoded_message_product[:5])

true_message_product = [m1 * m2 for m1, m2 in zip(message1, message2)]
print("True message product length:", len(true_message_product))
print("First 5 elements of true product:", true_message_product[:5])

assert len(decoded_message_product) == len(true_message_product), "Decoded message has incorrect length"
print("Multiplication is homomorphic:", np.allclose(decoded_message_product, true_message_product,  rtol=1e-2, atol=1e-1))  

NameError: name 'key_generator' is not defined

In [8]:
encrypted_poly_product = evaluator.multiply(encrypted_poly1, encrypted_poly2, relin_key)

PARALLEL!!!!!!
PARALLEL!!!!!!
PARALLEL!!!!!!
PARALLEL!!!!!!
PARALLEL!!!!!!
PARALLEL!!!!!!


In [9]:
encrypted_poly_product = evaluator.multiply(encrypted_poly1, encrypted_poly2, relin_key, is_parallel=False)

PARALLEL!!!!!!
PARALLEL!!!!!!
